# 日本語 BERT Base Model Fine-tuning & Deployment on Inferentia2
本 Notebook の元ネタのブログはこちらから
+ https://aws.amazon.com/jp/blogs/news/aws-trainium-amazon-ec2-trn1-ml-training-part1/
+ https://aws.amazon.com/jp/blogs/news/aws-trainium-amazon-ec2-trn1-ml-training-part2/

## 事前準備

In [1]:
%env TOKENIZERS_PARALLELISM=True #Supresses tokenizer warnings making errors easier to detect
!pip install -U pip
!pip install -U transformers[ja]==4.27.4 datasets

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [2]:
!pip list | grep "neuron\|torch"

aws-neuronx-runtime-discovery 2.9
libneuronxla                  0.5.391
neuron-cc                     1.15.0.0+eec0c3604
neuronperf                    1.8.0.0+5dccae385
neuronx-cc                    2.8.0.25+a3ad0f342
neuronx-hwm                   2.8.0.3+2b7c6da39
optimum-neuron                0.0.9.dev0
torch                         1.13.1
torch-neuron                  1.13.1.2.8.9.0
torch-neuronx                 1.13.1.1.9.0
torch-xla                     1.13.1+torchneuron8
torchmetrics                  1.0.1
torchvision                   0.14.1
transformers-neuronx          0.5.58
Note: you may need to restart the kernel to use updated packages.


In [3]:
!sudo rmmod neuron; sudo modprobe neuron

## データセットの準備
本テストでは、Huggingface Hub で利用可能な以下のセンチメント（感情）データセットのうち、日本語のサブセットを使用します。
https://huggingface.co/datasets/tyqiangz/multilingual-sentiments

本テストではテキストデータをPositiveかNegativeに分類する 2 クラスの分類問題として扱うことにします。元々のデータセットは positive(LABEL_0)、neutral(LABEL_1)、negative(LABEL_2)としてラベル付けされていますが、neutralのデータは使用しないこととし、ラベルをpositive(LABEL_0)、negative(LABEL_1)として再定義します。

In [4]:
from transformers import BertJapaneseTokenizer
from datasets import load_dataset

# Prepare dataset
dataset = load_dataset("tyqiangz/multilingual-sentiments", "japanese")
print(dataset)

print(dataset["train"].features)

dataset = dataset.remove_columns(["source"])
dataset = dataset.filter(lambda dataset: dataset["label"] != 1)
dataset = dataset.map(lambda dataset: {"labels": int(dataset["label"] == 2)}, remove_columns=["label"])

print(dataset["train"][20000])
print(dataset["train"][50000])

DatasetDict({
    train: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 120000
    })
    validation: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['text', 'source', 'label'],
        num_rows: 3000
    })
})
{'text': Value(dtype='string', id=None), 'source': Value(dtype='string', id=None), 'label': ClassLabel(names=['positive', 'neutral', 'negative'], id=None)}
{'text': '箱を開けただけで予想以上に匂いが甘くて 私には合わないかもです。 でも もったいないので今使ってるのが無くなったら使ってみます。', 'labels': 1}
{'text': '色々な変換アダプが売ってますけど長さも結構ありますし重宝しています。主にタブレットから→テレビに使ってるんですけど子供達に動画を見せる時タブレットを直接見せるよりテレビで大きな画面で見せた方が良いですね。なんの設定も要らないので直接繋げればすぐ使える所も気に入っています。', 'labels': 0}


次に、文章テキストのままだとモデルのトレーニングはできないため、テキストを意味のある単位で分割（トークナイズ）した上で数値に変換します。トークナイザーには MeCab ベースの BertJapaneseTokenizer を利用しました。

In [5]:
MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", max_length=128, truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

train_dataset = tokenized_datasets["train"].shuffle().select(range(4000))
eval_dataset = tokenized_datasets["test"].shuffle().select(range(256))

# Save dataset
train_dataset.save_to_disk("./train/")
eval_dataset.save_to_disk("./test/")

Saving the dataset (0/1 shards):   0%|          | 0/4000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/256 [00:00<?, ? examples/s]

実際にどのように変換されているのか、以下のスクリプトを実行し確認してみましょう。

In [6]:
index = 50000
print(dataset["train"][index])
print('Tokenize:', tokenizer.tokenize(dataset["train"]['text'][index]))
print('Encode:', tokenizer.encode(dataset["train"]['text'][index]))

{'text': '色々な変換アダプが売ってますけど長さも結構ありますし重宝しています。主にタブレットから→テレビに使ってるんですけど子供達に動画を見せる時タブレットを直接見せるよりテレビで大きな画面で見せた方が良いですね。なんの設定も要らないので直接繋げればすぐ使える所も気に入っています。', 'labels': 0}
Tokenize: ['色々', 'な', '変換', 'アダ', '##プ', 'が', '売っ', 'て', 'ます', 'けど', '長', 'さ', 'も', '結', '##構', 'あり', 'ます', 'し', '重宝', 'し', 'て', 'い', 'ます', '。', '主', 'に', 'タブレット', 'から', '→', 'テレビ', 'に', '使っ', 'てる', 'ん', 'です', 'けど', '子供', '達', 'に', '動画', 'を', '見せる', '時', 'タブレット', 'を', '直接', '見せる', 'より', 'テレビ', 'で', '大きな', '画面', 'で', '見せ', 'た', '方', 'が', '良い', 'です', 'ね', '。', 'なん', 'の', '設定', 'も', '要', '##ら', 'ない', 'ので', '直接', '繋', '##げ', '##れ', 'ば', 'すぐ', '使える', '所', 'も', '気に入っ', 'て', 'い', 'ます', '。']
Encode: [2, 16166, 18, 4618, 6568, 28526, 14, 18544, 16, 2610, 11218, 177, 26, 28, 224, 28902, 130, 2610, 15, 26736, 15, 16, 21, 2610, 8, 137, 7, 21024, 40, 2592, 571, 7, 2110, 7134, 1058, 2992, 11218, 1803, 784, 7, 4884, 11, 7237, 72, 21024, 11, 2031, 7237, 221, 571, 12, 1200, 4180, 12, 2685, 10, 283, 14, 3614, 2992, 1852, 8, 4508, 5, 1374, 28, 5

## Trainer API を使用した トレーニング（ファインチューニング）実行
Transformers には Trainer という便利なクラスがあり、Torch Neuron からも利用可能です。 ここでは Trainer API を利用してトレーニングを実行していきます。

### neuron_parallel_compile による事前コンパイル
トレーニングの各ステップでは、グラフがトレースされ、トレースされたグラフが以前のものと異なる場合は、再度計算グラフのコンパイルが発生します。大規模なモデルの場合、各グラフのコンパイル時間が長くなることがあり、トレーニング時間の中で占めるコンパイル時間がボトルネックとなってしまう場合もあり得ます。このコンパイル時間を短縮するため、PyTorch Neuron では neuron_parallel_compile ユーティリティが提供されています。neuron_parallel_compile は、スクリプトの試行からグラフを抽出し並列事前コンパイルを実施、コンパイル結果（NEFF : Neuron Executable File Format）をキャッシュとしてディスク上に保持します。

では実際に事前コンパイルを実行してみましょう。以下の内容でbert-jp-precompile.pyというファイル名の Python スクリプトを作成し実行します。スクリプトは基本的にこの後実行するトレーニングスクリプトと同じ内容ですが、neuron_parallel_compileはグラフの高速コンパイルのみを目的とし実際の演算は実行されず、出力結果は無効となります。トレーニング実行中も必要に応じてグラフはコンパイルされるため、この事前コンパイルのプロセスはスキップし、次のトレーニング処理に直接進んでも問題はありません。

コンパイル時間を短縮するためデータセット、epoch 数を制限している点にご注意ください。コンパイル結果は `/var/tmp/neuron-compile-cache/` 以下に保存されています。

In [7]:
%%writefile bert-jp-precompile.py

from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_from_disk
import torch, torch_xla.core.xla_model as xm
import os

os.environ["NEURON_CC_FLAGS"] = "--model-type=transformer"

device = xm.xla_device()

MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

train_dataset = load_from_disk("./train/").with_format("torch")
train_dataset = train_dataset.select(range(64))

eval_dataset = load_from_disk("./test/").with_format("torch")
eval_dataset = eval_dataset.select(range(64))

training_args = TrainingArguments(
    num_train_epochs = 2,
    learning_rate = 5e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    output_dir = "./results",
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
)

train_result = trainer.train()

Overwriting bert-jp-precompile.py


In [8]:
!time XLA_USE_BF16=1 neuron_parallel_compile python3 bert-jp-precompile.py

2023-08-14 08:09:07.000974:  715601  INFO ||PARALLEL_COMPILE||: Removing existing workdir /tmp/parallel_compile_workdir
2023-08-14 08:09:07.000977:  715601  INFO ||PARALLEL_COMPILE||: Running trial run (add option to terminate trial run early; also ignore trial run's generated outputs, i.e. loss, checkpoints)
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).

### シングルワーカーでのトレーニング実行
次に実際にトレーニングを実行してみます。事前コンパイルを実行した場合でも、追加のコンパイルが発生することがあります。一通りのコンパイルが終了した後、2度目以降の実行では、Neuron コアの恩恵を受けた高速トレーニングを体験できます。以下の内容で bert-jp-single.py というファイル名の Python スクリプトを作成し実行してみましょう。

先程の事前コンパイルとは異なり、今回は実際にトレーニングを実行するため、用意したデータセット全てに対して epoch = 10 で実行します。

In [9]:
%%writefile bert-jp-single.py

from transformers import BertForSequenceClassification, BertJapaneseTokenizer, Trainer, TrainingArguments
from datasets import load_from_disk
import torch, torch_xla.core.xla_model as xm
import os

os.environ["NEURON_CC_FLAGS"] = "--model-type=transformer"

device = xm.xla_device()

MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

train_dataset = load_from_disk("./train/").with_format("torch")
eval_dataset = load_from_disk("./test/").with_format("torch")

training_args = TrainingArguments(
    num_train_epochs = 10,
    learning_rate = 5e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    output_dir = "./results",
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer,
)

train_result = trainer.train()
print(train_result)

eval_result = trainer.evaluate()
print(eval_result)

trainer.save_model("./results")

Overwriting bert-jp-single.py


In [10]:
!time XLA_USE_BF16=1 python3 bert-jp-single.py

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialize

ステップ数 5000 のトレーニングが 9~10分程で完了しました。

トレーニング実行中に、AWS Neuron で提供される `neuron-top` ツールを利用すると、Neuron コア及び vCPU の利用率、アクセラレータメモリ、ホストメモリの利用状況等を確認することができます。inf2.xlarge には、一つの Inferentia2 チップ、チップ内に二つの Neuron コアが搭載されています。結果から、二つある Neuron コア（NC0 及び NC1）のうち一つの Neuron コアのみが利用されていることが分かります。まだ最適化の余地はありそうです。

生成されたモデルから期待通りの出力が得られるか確認しておきましょう。

In [11]:
from transformers import pipeline

classifier = pipeline("text-classification", model = "./results/")

print(classifier("大変すばらしい商品でした。感激です。"))
print(classifier("期待していた商品とは異なりました。残念です。"))

[{'label': 'LABEL_0', 'score': 0.9999897480010986}]
[{'label': 'LABEL_1', 'score': 0.9999812841415405}]


期待通りの出力を得られることが確認できたようです。

## torchrun を用いたマルチワーカーでのトレーニング実行
それでは、先程のトレーニングスクリプトに変更を加え、二つある Neuron コアを有効活用してみましょう。複数の Neuron コアを利用したマルチワーカーで実行するためには `torchrun` コマンドを利用します。`torchrun` コマンドに対して、オプション `--nproc_per_node` で利用する Neuron コアの数（並列実行するワーカー数）を指定します。trn1.2xlarge (inf2.xlarge) では 2 を、trn1.32xlargeの場合は 2, 8, 32 が指定可能です。

`torchrun` を利用したデータパラレルトレーニングを実行するにあたって、先程のスクリプトに一部変更を加えた `bert-jp-dual.py` というファイル名のスクリプトを作成し実行します。

それでは変更後のスクリプトを利用して　inf2.xlarge 上の二つ Neuron コアを利用したトレーニングを実行してみましょう。シングルワーカーでのトレーニング結果と比較し `Total train batch size` の値が倍の 16 に、`Total optimization steps` が半分の 2500 となっている点を確認できると思います。

シングルワーカー時の手順同様、まずは事前コンパイルを実行し、その後に実際のトレーニングを実行します。

In [12]:
%%writefile bert-jp-dual-precompile.py

from transformers import BertForSequenceClassification, BertJapaneseTokenizer, Trainer, TrainingArguments
from datasets import load_from_disk
import torch, torch_xla.distributed.xla_backend
import os

os.environ["NEURON_CC_FLAGS"] = "--model-type=transformer"

MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

train_dataset = load_from_disk("./train/").with_format("torch")
train_dataset = train_dataset.select(range(64))

eval_dataset = load_from_disk("./test/").with_format("torch")
eval_dataset = eval_dataset.select(range(64))


training_args = TrainingArguments(
    num_train_epochs = 2,
    learning_rate = 5e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    output_dir = "./results",
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
)

train_result = trainer.train()
print(train_result)

eval_result = trainer.evaluate()
print(eval_result)

Overwriting bert-jp-dual-precompile.py


In [13]:
!time XLA_USE_BF16=1 neuron_parallel_compile torchrun --nproc_per_node=2 bert-jp-dual-precompile.py

2023-08-14 08:24:16.000996:  904568  INFO ||PARALLEL_COMPILE||: Removing existing workdir /tmp/parallel_compile_workdir
2023-08-14 08:24:16.000998:  904568  INFO ||PARALLEL_COMPILE||: Running trial run (add option to terminate trial run early; also ignore trial run's generated outputs, i.e. loss, checkpoints)
*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weig

In [14]:
%%writefile bert-jp-dual.py

from transformers import BertForSequenceClassification, BertJapaneseTokenizer, Trainer, TrainingArguments
from datasets import load_from_disk
import torch, torch_xla.distributed.xla_backend
import os

os.environ["NEURON_CC_FLAGS"] = "--model-type=transformer"

MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

train_dataset = load_from_disk("./train/").with_format("torch")
eval_dataset = load_from_disk("./test/").with_format("torch")

training_args = TrainingArguments(
    num_train_epochs = 10,
    learning_rate = 5e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    output_dir = "./results",
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer,
)

train_result = trainer.train()
print(train_result)

eval_result = trainer.evaluate()
print(eval_result)

trainer.save_model("./results")

Overwriting bert-jp-dual.py


In [15]:
!time XLA_USE_BF16=1 torchrun --nproc_per_node=2 bert-jp-dual.py

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT ex

トレーニング実行中の neuron-top の出力も確認してみましょう。今度は二つの Neuron コアが利用されている事が確認できると思います。トレーニングに要する実行時間も 5~6 分に削減されました。

## 推論実行
先ほどは生成されたモデルから期待通りの出力が得られるかどうかCPU上で推論実行し、結果を確認しました。ここでは生成されたモデルをinf2.xlarge上で推論実行します。

In [16]:
import torch
import torch_neuronx
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import transformers

from transformers import BertForSequenceClassification, BertJapaneseTokenizer

def encode(tokenizer, *inputs, max_length=128, batch_size=1):
    tokens = tokenizer.encode_plus(
        *inputs,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )
    return (
        torch.repeat_interleave(tokens['input_ids'], batch_size, 0),
        torch.repeat_interleave(tokens['attention_mask'], batch_size, 0),
        torch.repeat_interleave(tokens['token_type_ids'], batch_size, 0),
    )

tokenizer = AutoTokenizer.from_pretrained("./results/")
model = AutoModelForSequenceClassification.from_pretrained("./results/", torchscript=True)


sequence = "大変すばらしい商品でした。感激です。"
paraphrase = encode(tokenizer, sequence)
cpu_paraphrase_logits = model(*paraphrase)[0]
print('CPU paraphrase logits:', cpu_paraphrase_logits.detach().numpy())

sequence = "期待していた商品とは異なりました。残念です。"
paraphrase = encode(tokenizer, sequence)
cpu_paraphrase_logits = model(*paraphrase)[0]
print('CPU paraphrase logits:', cpu_paraphrase_logits.detach().numpy())

CPU paraphrase logits: [[ 4.5357184 -4.789145 ]]
CPU paraphrase logits: [[-4.6640625  4.8995476]]


### Compile the model for Neuron

In [17]:
filename = 'model.pt'

In [18]:
model_neuron = torch_neuronx.trace(model, paraphrase)

# Save the TorchScript for inference deployment
torch.jit.save(model_neuron, filename)

### Run inference and compare results

In [19]:
model_neuron = torch.jit.load(filename)

sequence = "大変すばらしい商品でした。感激です。"
paraphrase = encode(tokenizer, sequence)
neuron_paraphrase_logits = model_neuron(*paraphrase)[0]
print('Neuron paraphrase logits:', neuron_paraphrase_logits.detach().numpy())

sequence = "期待していた商品とは異なりました。残念です。"
paraphrase = encode(tokenizer, sequence)
neuron_paraphrase_logits = model_neuron(*paraphrase)[0]
print('Neuron paraphrase logits:', neuron_paraphrase_logits.detach().numpy())

Neuron paraphrase logits: [[ 4.5336947 -4.7886157]]
Neuron paraphrase logits: [[-4.663004   4.9002094]]


CPUで推論実行した結果と同様の結果が得られている事が確認できました。推論性能を評価する方法は以下のサンプルをご参照下さい。
+ https://github.com/aws-neuron/aws-neuron-sdk/blob/master/src/examples/pytorch/torch-neuronx/bert-base-cased-finetuned-mrpc-inference-on-trn1-tutorial.ipynb